# Visualization of Planet-forming Disk

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, LogNorm

import volrender

Read **THOMAS PFEILS** data. Data is currently (until 02.02.2022) available [here](https://gigamove.rwth-aachen.de/de/download/ebdbc12c85438bd224a5015206840893).

In [ ]:
with np.load('../data/pluto_data.npz') as f:
    data = f['rho']

In [ ]:
vmax = data.max()
lognorm = LogNorm(vmin=vmax * 1e-4, vmax=vmax, clip=True)
datacube = lognorm(data.ravel()).reshape(data.shape).data

In [ ]:
ren = volrender.Renderer(datacube)
colors = ren.transferfunction.colors.copy()

ren.transferfunction.x0            = [0.1, 0.4, 0.9]
ren.transferfunction.sigma         = [0.03, 0.03, 0.05]
ren.transferfunction.colors        = colors
ren.transferfunction.colors[:, -1] = [0.1, 0.2, 0.1]
ren.transferfunction.A             = [0.2, 0.1, 0.3]

In [ ]:
ren.render(0, 0)

## Test Interpolation

In [ ]:
ren.init_interpolation()

In [ ]:
%%time
ren._f_int = False
ren.render(10, 0)
res_p = ren.data_obs.copy()

In [ ]:
%%time
ren._f_int = True
ren.render(10, 0)
res_f = ren.data_obs.copy()

In [ ]:
x = np.linspace(ren.x[0], ren.x[-1], ren._N)
y = np.linspace(ren.y[0], ren.y[-1], ren._N)
z = np.linspace(ren.z[0], ren.z[-1], ren._N)

_x = 150
_y = 245
_z = 0

ix = x.searchsorted(_x)
iy = y.searchsorted(_y)
iz = z.searchsorted(_z)

vmax = np.max(ren.data)
vmin = 0
norm = Normalize(vmin=vmin, vmax=vmax)
#norm = LogNorm(vmin=1e-4 * vmax, vmax=vmax)

f, axs = plt.subplots(1, 3, figsize=(15,4), sharex=True, sharey=True)
for ax in axs:
    ax.set_aspect('equal')
axs[0].pcolormesh(ren.x, ren.y, datacube[:, :, datacube.shape[-1] // 2].T, norm=norm)
axs[1].pcolormesh(x, y, res_p[:, :, iz].T, norm=norm)
axs[2].pcolormesh(x, y, res_f[:, :, iz].T, norm=norm, lw=0.001, ec='w')
axs[2].set_xlim(0, 200)
axs[2].set_ylim(80, 250)

for ax in axs:
    ax.plot(x[ix], y[iy], 'rx')

## Plots

In [ ]:
f, ax = ren.plot(norm=lognorm, diagnostics=True)

Same disk different transfer function

In [ ]:
ren.transferfunction.x0     = np.array([0.2, 0.88])
ren.transferfunction.sigma  = np.array([0.05, 0.02])
ren.transferfunction.colors = np.array([
    [1.  , .5  , 0.  , 0.1],
    [.25  , .25  , .75  , 0.2]])
ren.render(update=False)
ren.plot(diagnostics=False)

## Make a movie

due to the size, here we can just use 1 CPU or we get memory issues

In [ ]:
n_ang = 90
theta = np.linspace(45, 45, n_ang)
phi = np.linspace(0, 180, n_ang)
fname = 'movie_pluto.mp4'
volrender.render_helper.render_movie(ren.data, theta, phi, ncpu=1, tf=ren.transferfunction, fname=fname)

In [ ]:
from IPython.display import HTML
HTML(f"""
<video width="500" controls>
  <source src="{fname}" type="video/mp4">
</video>
""")